## Import libraries and load data

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree, KDTree
import geopandas as gpd
from shapely.geometry import Point

### Load files and create radian columns

In [2]:
optd=pd.read_csv(r"data/optd-airports-sample.csv.gz")
for column in optd[["latitude", "longitude"]]:
    rad = np.deg2rad(optd[column].values)
    optd[f'{column}_rad'] = rad
optd.head()

,iata_code,latitude,longitude,latitude_rad,longitude_rad
0,AAA,-17.352606,-145.509956,-0.302860,-2.539628
1,AAB,-26.693170,141.047800,-0.465884,2.461749
2,AAC,31.073330,33.835830,0.542332,0.590547
3,AAD,6.096820,46.638250,0.106410,0.813991
4,AAE,36.822225,7.809167,0.642669,0.136296


In [3]:
geo_samples=pd.read_csv(r"data/user-geo-sample.csv.gz")
for column in geo_samples[["geoip_latitude", "geoip_longitude"]]:
    rad = np.deg2rad(geo_samples[column].values)
    geo_samples[f'{column}_rad'] = rad
geo_samples.head()

,uuid,geoip_latitude,geoip_longitude,geoip_latitude_rad,geoip_longitude_rad
0,DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B,-37.833302,145.050003,-0.660316,2.531600
1,DAEF2221-14BE-467B-894A-F101CDCC38E4,52.516701,4.666700,0.916589,0.081449
2,31971B3E-2F80-4F8D-86BA-1F2077DF36A2,35.685001,139.751404,0.622821,2.439122
3,1A29A45C-D560-43D8-ADAB-C2F0AD068FFE,44.840401,-0.580500,0.782613,-0.010132
4,A6EC281B-B8EC-465A-8933-F127472DB0A3,51.963299,4.499700,0.906931,0.078535


In [4]:
print(geo_samples.shape,optd.shape)

(1000000, 3) (9227, 3)


## Define clustering algorithm

In [38]:
# Takes the first group's latitude and longitude values to construct
# the kd tree.
kd = KDTree(optd[["latitude_rad", "longitude_rad"]].values, metric='euclidean')
# The amount of neighbors to return.
number_initial_neightbors = 5
# The amount of neighbors to return.
number_final_neightbors = 1

## Define function to find closest airport

In [80]:
def closest_airport_v1(uuid,geo_sample_latitude,geo_sample_longitude):
    """_summary_

    Args:
        geo_sample (_type_): _description_
    """
    data=np.array([[geo_sample_latitude,geo_sample_longitude]])
    distances, indices_euclidean = kd.query(data, k = number_initial_neightbors)
    ball = BallTree(optd.loc[indices_euclidean[0]][["latitude_rad", "longitude_rad"]].values, metric='haversine')
    distances, indices = ball.query(data, k = number_final_neightbors)
    
    indice=indices_euclidean[0][indices[0]]
    # print(indice)
    return uuid,optd.loc[indice[0]]['iata_code']

In [92]:
ball = BallTree(optd[["latitude_rad", "longitude_rad"]].values, metric='haversine')
def closest_airport_v2(uuid,geo_sample_latitude,geo_sample_longitude):
    """_summary_

    Args:
        uuid (str): unique_sample_identifier
        geo_sample_latitude (int): latitude in radians
        geo_sample_longitude (int): longitude in radians

    Returns:
        tuple: uuid,iaat_code
    """
    data=np.array([[geo_sample_latitude,geo_sample_longitude]])
    distances, indices = ball.query(data, k = number_final_neightbors)
    return uuid,optd.loc[indices[0][0]]['iata_code']

In [129]:
ball = BallTree(optd[["latitude_rad", "longitude_rad"]].values, metric='haversine')
def closest_airport_v3(geo_sample):
    """_summary_

    Args:
        uuid (str): unique_sample_identifier
        geo_sample_latitude (int): latitude in radians
        geo_sample_longitude (int): longitude in radians

    Returns:
        tuple: uuid,iaat_code
    """
    distances, indices = ball.query(geo_sample[['geoip_latitude_rad','geoip_longitude_rad']], k = number_final_neightbors)
    # print(indices.flatten())
    return pd.DataFrame(geo_sample['uuid']).join(optd.loc[indices.flatten()]['iata_code'].reset_index(drop=True))

In [61]:
closest_airport('DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B',-0.6603156788459609, 2.531600133280997)

[4648 4702 4712  464 1081]


('DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B', 'MBW')

In [82]:
geo_samples.head(1000).apply(lambda x: closest_airport_v1(x['uuid'],x['geoip_latitude_rad'],x['geoip_longitude_rad']),axis=1)

0      (DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B, MBW)
1      (DAEF2221-14BE-467B-894A-F101CDCC38E4, AMS)
2      (31971B3E-2F80-4F8D-86BA-1F2077DF36A2, HND)
3      (1A29A45C-D560-43D8-ADAB-C2F0AD068FFE, BOD)
4      (A6EC281B-B8EC-465A-8933-F127472DB0A3, RTM)
                          ...                     
995    (C195AD1A-EEF3-4188-95D7-53ED43BE7DC8, RAO)
996    (F5F979CB-1A71-446F-BDA7-CB720EDFC8B4, EDM)
997    (5A863C48-F6C4-411E-BAA4-21BC8D97127B, SVG)
998    (CAC6F5A1-6C3A-4186-A34A-8D9574EAE4C7, ZRH)
999    (8EE6BDAB-2CA0-4345-970F-E2A191FC2036, ORY)
Length: 1000, dtype: object

In [105]:
geo_samples.head(1000).apply(lambda x: closest_airport_v2(x['uuid'],x['geoip_latitude_rad'],x['geoip_longitude_rad']),axis=1)

0      (DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B, MBW)
1      (DAEF2221-14BE-467B-894A-F101CDCC38E4, AMS)
2      (31971B3E-2F80-4F8D-86BA-1F2077DF36A2, HND)
3      (1A29A45C-D560-43D8-ADAB-C2F0AD068FFE, BOD)
4      (A6EC281B-B8EC-465A-8933-F127472DB0A3, RTM)
                          ...                     
995    (C195AD1A-EEF3-4188-95D7-53ED43BE7DC8, RAO)
996    (F5F979CB-1A71-446F-BDA7-CB720EDFC8B4, EDM)
997    (5A863C48-F6C4-411E-BAA4-21BC8D97127B, SVG)
998    (CAC6F5A1-6C3A-4186-A34A-8D9574EAE4C7, ZRH)
999    (8EE6BDAB-2CA0-4345-970F-E2A191FC2036, ORY)
Length: 1000, dtype: object

In [95]:
geo_samples.columns

Index(['uuid', 'geoip_latitude', 'geoip_longitude', 'geoip_latitude_rad',
       'geoip_longitude_rad'],
      dtype='object')

In [130]:
closest_airport_v3(geo_samples.head(1000)[['uuid', 'geoip_latitude_rad','geoip_longitude_rad']])

,uuid,iata_code
0,DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B,MBW
1,DAEF2221-14BE-467B-894A-F101CDCC38E4,AMS
2,31971B3E-2F80-4F8D-86BA-1F2077DF36A2,HND
3,1A29A45C-D560-43D8-ADAB-C2F0AD068FFE,BOD
4,A6EC281B-B8EC-465A-8933-F127472DB0A3,RTM
...,...,...
995,C195AD1A-EEF3-4188-95D7-53ED43BE7DC8,RAO
996,F5F979CB-1A71-446F-BDA7-CB720EDFC8B4,EDM
997,5A863C48-F6C4-411E-BAA4-21BC8D97127B,SVG
998,CAC6F5A1-6C3A-4186-A34A-8D9574EAE4C7,ZRH


In [16]:
# Takes the first group's latitude and longitude values to construct
# the kd tree.
kd = KDTree(optd[["latitude_rad", "longitude_rad"]].values, metric='euclidean')
# The amount of neighbors to return.
k = 5
# Executes a query with the second group. This will return two
# arrays.


array([[4648, 4702, 4712,  464, 1081]], dtype=int64)

In [28]:
distances, indices = kd.query(geo_samples[["geoip_latitude_rad", "geoip_longitude_rad"]].head(1), k = number_neightbors)
ball = BallTree(optd.loc[indices[0]][["latitude_rad", "longitude_rad"]].values, metric='haversine')
# The amount of neighbors to return.
k = 1
# Executes a query with the second group. This will return two
# arrays.
distances, indices = ball.query(geo_samples[["geoip_latitude_rad", "geoip_longitude_rad"]].head(1), k = k)
indices

array([[0, 1, 2, 3, 4]], dtype=int64)

In [75]:
ball = BallTree(optd[["latitude_rad", "longitude_rad"]].values, metric='haversine')
distances, indices = ball.query(geo_samples[["geoip_latitude_rad", "geoip_longitude_rad"]].head(1), k = number_final_neightbors)
indices

array([[4648]], dtype=int64)

In [32]:
geo_samples.head(1).values[0]

array(['DDEFEBEA-98ED-49EB-A4E7-9D7BFDB7AA0B', -37.83330154418945,
       145.0500030517578, -0.6603156788459609, 2.531600133280997],
      dtype=object)

In [6]:
# Takes the first group's latitude and longitude values to construct
# the kd tree.
ball = BallTree(optd[["latitude_rad", "longitude_rad"]].values, metric='haversine')
# The amount of neighbors to return.
k = 2
# Executes a query with the second group. This will return two
# arrays.
distances, indices = ball.query(geo_samples[["geoip_latitude_rad", "geoip_longitude_rad"]], k = k)
indices

array([[4648, 4702],
       [ 268, 1795],
       [3022, 5684],
       ...,
       [ 691, 7202],
       [6725,  268],
       [ 828,  363]], dtype=int64)

In [5]:
indices.shape

(1000000, 2)

In [5]:

# create a GeoDataFrame from the dataframe and the point geometries
geometry = [Point(xy) for xy in zip(optd['longitude'], optd['latitude'])]
gdf = gpd.GeoDataFrame(optd, geometry=geometry, crs="EPSG:4326")

# create a spatial index for the GeoDataFrame
gdf_sindex = gdf.sindex
def calcular_aeropuerto_cercano(longitude,latitude):
    point=Point(longitude,latitude)
    # find the index of the nearest point to the given point
    nearest_idx = list(gdf_sindex.nearest(point.bounds, 1))[0]
    # retrieve the nearest point from the GeoDataFrame
    nearest_point = gdf.iloc[nearest_idx]
    # print the identifier of the nearest point
    print(nearest_point['iata_code'])


In [6]:
calcular_aeropuerto_cercano(-145.509956,-17.352606)

TypeError: One of the arguments is of incorrect type. Please provide only Geometry objects.